### Load dataset and rename columns
> Note: **Only AMST** are contained in df. HNK products where filtered.

In [86]:
import pandas as pd
import numpy as np
from re import sub

def column_name_to_snake_case(s):
    s = sub(r"[\/( \- ) ]", '_', s)
    s = sub(r"(___)|(__)", '_', s).lower()
    return s if s[-1] != '_' else s[:-1]

df = pd.read_csv(r'..\data\raw\Heineken - Data Science Use Case.csv', parse_dates=['Date/Time'])
df = (df.drop(columns=df.columns[0])
        .rename(columns={col:column_name_to_snake_case(col) for col in df.columns[1:]})
        .query('product == "AMST"')
     )

### Rows with null values

In [87]:
nulls = df[pd.isnull(df).any(axis=1)]

#### Analyzing color null values
All color null values have no other null value besides the target column.
> They will be used as a test dataset

In [93]:
target = nulls.query('color.isnull()')
target.drop('color', axis=1).isnull().sum(axis=0)

# saving to parse dtypes faster later 
dtypes = {col:target[col].dtype for col in target.columns}

target.to_csv(r'..\data\test\test_data.csv', index=False)

